In [ ]:
import tensorflow as tf
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import TFBertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

pos_texts = pd.read_csv('positive.csv', encoding='utf8', sep=';', header=None)
neg_texts = pd.read_csv('negative.csv', encoding='utf8', sep=';', header=None)

In [3]:
pos_texts.sample(10)

0           1               2   \
46444   409998247692738560  1386586174  yulenka_masyuk   
56215   410089110703849474  1386607837       MsWildFan   
80149   410740771566796801  1386763205      fagabazyzo   
51810   410049235761897472  1386598330      like_merry   
76930   410703427375595520  1386754302       Donna_132   
57895   410127228429541376  1386616925       WlasovaAs   
19782   409407651294871552  1386445365     zlaya_matka   
28959   409620860790919168  1386496198     L_Shatalova   
51598   410047317669322752  1386597873        Ronny_n7   
107245  411154477052608512  1386861840       inesa0072   

                                                       3   4   5   6   7   \
46444   настроение нет вообще.. хотя за окном красота)...   1   0   0   0   
56215   @DarrenCrizzz пфф да легко)в моих мечтах он да...   1   0   0   0   
80149   RT @Msurmach: @__Mila_ya а вы ездили погулять ...   1   0   2   0   
51810   @_levianee_ так звучит, как будто скоро апокал...   1   0   0   0   
76930   И ваще политика была не по плану! Должен был б...   1   0   0   0   
57895   @OlesyaSonFlogi ООУУ,милашества^^\nЯ тебя тоже...   1   0   0   0   
19782   @MuseOfdaySystem как финн и джейк с:\nдавай по...   1   0   0   0   
28959   Я в ЦМИ на мотивационной школе. Занятный у них...   1   0   0   0   
51598   @brishka_gmc @fixer7sk это один наш торрент-тр...   1   0   0   0   
107245  Самое красивое новогоднее фото) http://t.co/xO...   1   0   0   0   

           8    9    10  11  
46444     706   17   28   0  
56215    6001  157  122   3  
80149     640  122   89   0  
51810   31297   73  124   0  
76930    8781   36   67   0  
57895      34   32   20   0  
19782    2429  236  187   0  
28959   11710  688   88  24  
51598    5144   50  152   0  
107245    167  144   19   0

Обратите внимание на специальные токены [CLS] и [SEP], которые мы добавляем в началои конец предложения.

In [4]:
sentences = np.concatenate([pos_texts[3].values, neg_texts[3].values])

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = [[1] for _ in range(pos_texts.shape[0])] + [[0] for _ in range(neg_texts.shape[0])]

In [5]:
assert len(sentences) == len(labels) == pos_texts.shape[0] + neg_texts.shape[0]

In [6]:
print(sentences[1000])

[CLS] Дим, ты помогаешь мне, я тебе, все взаимно, все правильно) [SEP]


In [7]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_gt, test_gt = train_test_split(sentences, labels, test_size=0.3)

In [8]:
print(len(train_gt), len(test_gt))

158783 68051


Проводим токинезацию по бертовски!

In [9]:
%%time
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
print (tokenized_texts[0])

['[CLS]', '@', 'poly', '##a', '_', 'from', '_', 'mars', 'о', '##п', 'о', '##п', 'к', '##л', '##а', '##с', '##с', '##н', '##о', ',', 'а', 'у', 'м', '##е', '##н', '##я', 'е', '##щ', '##е', 'и', '##д', '##у', '##т', ':', '-', '(', '[SEP]']
Wall time: 1min 3s


BERT'у нужно предоставить специальный формат входных данных.


- **input ids**: последовательность чисел, отождествляющих каждый токен с его номером в словаре.
- **labels**: вектор из нулей и единиц. В нашем случае нули обозначают негативную эмоциональную окраску, единицы - положительную.
- **segment mask**: (необязательно) последовательность нулей и единиц, которая показывает, состоит ли входной текст из одного или двух предложений. Для случая одного предложения получится вектор из одних нулей. Для двух: <length_of_sent_1> нулей и <length_of_sent_2> единиц.
- **attention mask**: (необязательно) последовательность нулей и единиц, где единицы обозначают токены предложения, нули - паддинг.


Паддинг нужен для того, чтобы BERT мог работать с предложениями разной длины. Выбираем максимально возможную длину предложения (в нашем случае пусть это будет 100). 

Теперь более длинные предложения будем обрезать до 100 токенов, а для более коротких использовать паддинг. Возьмем готовую функцию `pad_sequences` из библиотеки `keras`.

In [10]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(
    input_ids,
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)

attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [11]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, train_gt, 
    random_state=42,
    test_size=0.1
)

train_masks, validation_masks, _, _ = train_test_split(
    attention_masks,
    input_ids,
    random_state=42,
    test_size=0.1
)

Теперь подробнее рассмотрим процесс файн-тюнинга. Как мы помним, первый токен в каждом предложении - это [CLS]. В отличие от скрытого состояния, относящего к обычному слову (не метке [CLS]), скрытое состояние относящееся к этой метке должно содержать в себе аггрегированное представление всего предложения, которое дальше будет использоваться для классификации. Таким образом, когда мы скормили предложение в процессе обучения сети, выходом будет вектор со скрытым состоянием, относящийся к метке [CLS]. Дополнительный полносвязный слой, который мы добавили, имеет размер [hidden_state, количество_классов], в нашем случае количество классов равно двум. То есть нав выходе мы получим два числа, представляющих классы "положительная эмоциональная окраска" и "отрицательная эмоциональная окраска".
Процесс дообучения достаточно дешев. По факту мы тренируем наш верхний слой и немного меняем веса во всех остальных слоях в процессе, чтобы подстроиться под нашу задачу.
Иногда некоторые слои специально "замораживают" или применяют разные стратегии работы с learning rate, в общем, делают все, чтобы сохранить "хорошие" веса в нижних слоях и ускорить дообучение. В целом, замораживание слоев BERTа обычно не сильно сказывается на итоговом качестве, однако надо помнить о тех случаях, когда данные, использованные для предобучения и дообучения очень разные (разные домены или стиль: академическая и разговорная лексика). В таких случаях лучше тренировать все слои сети, не замораживая ничего.
Загружаем BERT. bert-base-uncased - это версия "base" (в оригинальной статье рассказывается про две модели: "base" vs "large"), где есть только буквы в нижнем регистре ("uncased").

In [12]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_inputs, np.array(train_labels) ,epochs=2,batch_size=20,verbose=2)

Train on 142904 samples
Epoch 1/2


KeyboardInterrupt: 

In [14]:
preds = model.predict(validation_inputs[0:100])

In [15]:
preds = np.argmax(preds,axis=1)

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
accuracy_score(preds,np.array(validation_labels[0:100]))

0.99

In [ ]:
(прототип тетрадки все еще из лекции МФТИ)